In [7]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


class grade(BaseModel):
    """관련성 검사를 위한 이진 점수."""

    binary_score: str = Field(
        description="relevant or not relevant. Answer 'yes' if the answer is relevant to the document else answer 'no'"
    )


# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# 파서
output_parser = PydanticOutputParser(pydantic_object=grade)

# 프롬프트
prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {context} \n\n
    Here is the answer: {answer} \n
    If the document contains keyword(s) or semantic meaning related to the user answer, grade it as relevant. \n
    
    #Format of the answer: \n
    {format_instructions}""",
    input_variables=["context", "answer"],
)

# 체인
chain = (
    prompt.partial(format_instructions=output_parser.get_format_instructions())
    | model
    | output_parser
)

In [8]:
chain.invoke(
    {
        "context": "대한민국의 수도는 서울이다.",
        "answer": "제 생각엔 대한민국의 수도는 부산입니다.",
    }
)

grade(binary_score='no')

In [10]:
chain.invoke(
    {
        "context": "대한민국의 수도는 서울이다.",
        "answer": "대한민국(South Korea) 수도: 서울",
    }
)

grade(binary_score='yes')